<a href="https://colab.research.google.com/github/wajihh/genai_projects/blob/main/rag_gpt4o_faa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Required Libraries

In [ ]:
!pip install qdrant-client langchain transformers pypdf python-docx openai python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.28.3 which is incompatible.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=

#Load Envoirnment Variables

In [ ]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env")

# Get API key
openai_api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# Print to confirm (only for testing, avoid in production)
print(f"API Key Loaded: {openai_api_key[:5]}...")  # Just print first few chars


API Key Loaded: sk-pr...


# Upload pdf file

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files
from PyPDF2 import PdfReader

# Upload the PDF file
uploaded = files.upload()
# Process file
file_name = list(uploaded.keys())[0]
print(f"Uploaded file: {file_name}")

# Extract text from PDF
reader = PdfReader(file_name)
text = ""
for page in reader.pages:
    text += page.extract_text()

print(f"Extracted {len(text)} characters from the document.")


Saving eb-105-vertiports.pdf to eb-105-vertiports.pdf
Uploaded file: eb-105-vertiports.pdf
Extracted 90048 characters from the document.


#Step 3 Split Documents into Chunks

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# Initialize text splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=50
)

# Split the extracted text into chunks
text_chunks = text_splitter.split_text(text)
print(f"Document split into {len(text_chunks)} chunks.")


Document split into 201 chunks.


#Step 4 Step 4: Generate Embeddings Using a Pre-trained Model

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15


In [ ]:
!pip install sentence-transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load HuggingFace embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Generate embeddings for each chunk
embeddings = [embedding_model.embed_query(chunk) for chunk in text_chunks]
print(f"Generated embeddings for {len(embeddings)} chunks.")


<ipython-input-16-db66f38d670e>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generated embeddings for 201 chunks.


\

Step 5: Create a Vector Index in Qdrant

In [ ]:
# Load .env file
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env.qdrant")

True

In [ ]:
from qdrant_client import QdrantClient

# Initialize Qdrant client (replace with your Qdrant URL and API key)
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

# Create a collection in Qdrant
collection_name = "faa_document_collection"
vector_size = len(embeddings[0])  # Size of each vector from embedding

client.recreate_collection(
    collection_name=collection_name,
    vectors_config={"size": vector_size, "distance": "Cosine"}
)

# Upload document chunks and their embeddings to Qdrant
for i, embedding in enumerate(embeddings):
    client.upload_collection(
        collection_name=collection_name,
        vectors=[embedding],
        payload=[{"text": text_chunks[i], "chunk_index": i}]
    )

print(f"Uploaded {len(embeddings)} chunks to Qdrant.")


<ipython-input-19-a780a0ee12f6>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Uploaded 201 chunks to Qdrant.


#Step 6: Query and Retrieve Relevant Chunks

In [ ]:
def query_qdrant(query_text):
    # Generate embedding for the query
    query_embedding = embedding_model.embed_query(query_text)

    # Perform similarity search in Qdrant
    results = client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=3  # Retrieve top 3 matching chunks
    )

    # Display results
    for result in results:
        print(f"Chunk {result.payload['chunk_index']}:\n{result.payload['text']}\n")

# Example query input from the user
user_query = input("Enter your query: ")
query_qdrant(user_query)


Enter your query: what guidelines are contained in it?
Chunk 160:
conditions.  
 NFPA 400 , Hazardous Materials Code : Covers the minimum NFPA standards for 
the storage and handling of hazardous materials such as lithium batteries.  
 NFPA 418 , Standard for Heliports : This standard establishes fire safety standards 
for operations of heliports and rooftop hangars for the  protection of people, aircraft, 
and other property.  Future editions of this standard will include electric mobility 
asset considerations.

Chunk 13:
performance -based vertiport design AC is developed.  The guidance herein is not legally 
binding in its own right and will not be relied upon by the FAA as a separate basis for 
affirmative enforceme nt action or other administrative penalty.  Conformity with this 
guidance, as distinct from existing statutes, regulations, and grant assurances, is voluntary 
only, and nonconformity will not affect existing rights and obligations.  The standards

Chunk 25:
2.4. Sa

#Step 7: Synthesize the Response with GPT-4-mini